In [1]:
from __future__ import print_function    # (at top of module)
import sys
import time

import json
import requests
from props.default import *


# Miscellaneous Functions

In [2]:
def log(m):
        print(m)
        
def setToken(t):
    global DL_TOKEN 
    DL_TOKEN = { 'Authorization' : 'DirectLogin token=%s' % t}
    
def mergeHeaders(x, y):
    z = x.copy()
    z.update(y)
    return z

def merge_headers_list(header_list):
    output_dict = {}
    for header in header_list:
        output_dict.update(header)
    return output_dict
        

CONTENT_JSON  = { 'content-type'  : 'application/json' }

# 1. Login in

In [3]:
def login(username, password, consumer_key):
    login_url = '{0}/my/logins/direct'.format(BASE_URL)
    login_header  = { 'Authorization' : 'DirectLogin username="%s",password="%s",consumer_key="%s"' % (username, password, consumer_key)}
    # Login and receive authorized token
    log('Login as {0} to {1}'.format(login_header, login_url))
    r = requests.post(login_url, headers=login_header)
    if (r.status_code != 201):
        log("error: could not login")
        log("text: " + r.text)
        return r.text
    t = r.json()['token']
    log("Received token: {0}".format(t))
    setToken(t)
    return t, login_header

In [4]:
token, login_header = login('', '', '')


Login as {'Authorization': 'DirectLogin username="",password="",consumer_key=""'} to https://bnpparibas-irb.openbankproject.com/my/logins/direct
error: could not login
text: {"error":"OBP-20002: These DirectLogin parameters are missing: username, password, consumer_key"}


ValueError: too many values to unpack (expected 2)

# 2. Data Warehouse

In [309]:
def request_datawarehouse(field):
    post_data = {
        "size" : 20,
        "query"   : {"match_all":{          }  },
        "search_after" : [50],
        "sort": [{"customre_id":"asc"}]
    }
    # Send post request with attached json
#     param_scroll = {"scroll":"1m"}
    response = requests.post(u"{0}/obp/{1}/search/warehouse/{2}".format(BASE_URL, API_VERSION, field), json=post_data, headers=mergeHeaders(DL_TOKEN, CONTENT_JSON))
    return response.json()

In [303]:
def request_datawarehouse_agg():
    post_data = {
         "size" : 20,
        "aggs": {
            "byindex": {
                  "terms": {
                    "field": "_index"
                  }
                }
        },
        "query"   : {"match_all":{          }  },
    }
    # Send post request with attached json
    response = requests.post(u"{0}/obp/{1}/search/warehouse/{2}".format(BASE_URL, API_VERSION, 'ALL'), json=post_data, headers=mergeHeaders(DL_TOKEN, CONTENT_JSON))
    return response.json()